## Introduction
- A major part of energy system transformation is the transition of personal transport towards electric vehicles (EV)
- Meanwhile this faces a classical chicken or egg problem as market acceptance is based on available (public) charging infrastructure while efficient allocation of resccources to those infrastructure depends on enough users [](https://doi.org/10.1016/j.retrec.2020.100837)
- In Germany a great part of the populations have not the possiblity to install private charging devieces.
- Lots of people dont life in houses with carport or garages and even if a major share does not own their living property
- For making EV a viable alternative germany needs enough public charging stations

## Charging infrastructure in germany
- Gouverment see privat sector in charge to invest
- at the moment about 80 to 90 thousand charging points
- goal is 1 million until 2030 [](https://www.bundesregierung.de/breg-de/suche/ladepunkte-in-deutschland-1884666)
- Therefore the gouverment provides financial support for investors [](https://www.bundesregierung.de/breg-de/suche/ladepunkte-in-deutschland-1884666)

<-image here->
## Growing demand
- As this report by Roland Berger points out: While the energy transition proceeds more and more electric cars are on the roads 
- See image: The European light EV fleet should grow by 34% per year until 2050, reaching 227 million vehicles
- At the same time numerous competitors try to find valubale business models to to match the growing demand of public infrastructure.
- the main problem is here that EV drivers are (yet) highly unpredictable in their charging behaviors
- The profitablity of charging stations depends highly on workload which is uneaven distributed [](https://doi.org/10.3390/wevj8040936)
- Workload of german chargign stations is tracked in real time by the [Leitstelle Ladeinfrastruktur](https://nationale-leitstelle.de/verstehen/) (Charging infrastructure control centre) and ranges usually betwen 5% and 15%

<-diagram here->
## How to match the demand
- for eficient ressource allocation well informed location determination is crucial
- therefore we need:
    + Predict the regional demand of public charging infrastructure based on:
        - available charging stations
        - private charging opportunities
        - amount of cars
        - traveled distances
        - consumption per distance

    + Predict usage behavior of different charger types based on small area factors:
        - Traffic routes
        - Traffic volatility
        - length of stay
        - housing structure
        - periodic fluctuations



In [ ]:
from data_handler import SQLiteFetcher
from map_drawer import DrawMap as dm

with SQLiteFetcher('../../ChargeApp.db', kreisid=118) as fetcher:
    geo = fetcher.fetch_geometry_data("geometry")
    envelope = fetcher.fetch_kreise()[0].get("envelope")
    stations = fetcher.fetch_stations()

def center(envelope):
    """Find the center coordinates of an envelope."""
    clean_str = envelope.strip("{}")
    str_values = clean_str.split(", ")
    envelope = [float(x) for x in str_values]

    min_x = envelope[0]
    min_y = envelope[1]
    max_x = envelope[2]
    max_y = envelope[3]

    center_x = (min_x + max_x) / 2
    center_y = (min_y + max_y) / 2
    return center_x, center_y

dm = dm()
station_map = dm.plot_regions(geo)
station_map = dm.add_stations(stations)

center_x, center_y = center(envelope)

# Modify the layout to focus on a particular area
station_map['layout']['mapbox']['center']['lat'] = center_y
station_map['layout']['mapbox']['center']['lon'] = center_x
station_map['layout']['width'] = 1000  # in pixels
station_map['layout']['height'] = 1000  # in pixels
station_map['layout']['mapbox']['zoom'] = 10.7

station_map.write_image('frankfurt_map.jpeg')

In [ ]:
from data_handler import SQLiteFetcher
from map_drawer import DrawMap as dm

with SQLiteFetcher('../../ChargeApp.db', kreisid=179) as fetcher:
    geo = fetcher.fetch_geometry_data("geometry")
    envelope = fetcher.fetch_kreise()[0].get("envelope")
    stations = fetcher.fetch_stations()

def center(envelope):
    """Find the center coordinates of an envelope."""
    clean_str = envelope.strip("{}")
    str_values = clean_str.split(", ")
    envelope = [float(x) for x in str_values]

    min_x = envelope[0]
    min_y = envelope[1]
    max_x = envelope[2]
    max_y = envelope[3]

    center_x = (min_x + max_x) / 2
    center_y = (min_y + max_y) / 2
    return center_x, center_y

# Create an instance of DrawMap
dm = dm()
station_map = dm.plot_regions(geo)
#station_map = dm.add_envelope(envelope)
station_map = dm.add_stations(stations)

center_x, center_y = center(envelope)

# Modify the layout to focus on a particular area
station_map['layout']['mapbox']['center']['lat'] = center_y
station_map['layout']['mapbox']['center']['lon'] = center_x
station_map['layout']['width'] = 1000  # in pixels
station_map['layout']['height'] = 1000  # in pixels
station_map['layout']['mapbox']['zoom'] = 10.7

station_map.write_image('stuttgart_map.jpeg')


In [ ]:
from data_handler import SQLiteFetcher
import json

def to_geojson(kreis, geo):
    # Initialize the GeoJSON object
    geojson = {
        "type": "FeatureCollection",
        "features": []
    }
    
    for k, g in zip(kreis, geo):
        if k['KREISID'] == g['KREISID']:
            feature = {
                "type": "Feature",
                "properties": {},
                "geometry": {
                    "type": "Polygon",
                    "coordinates": [g['geometry']['rings']]
                }
            }
            
            # Copy metadata to properties
            for key, value in k.items():
                feature['properties'][key] = value
            
            geojson['features'].append(feature)
    
    return geojson

with SQLiteFetcher('../../ChargeApp.db', kreisid=179) as fetcher:
    kreis = fetcher.fetch_kreise()
    geo = fetcher.fetch_geometry_data("geometry")
    with open("stuttgart.geojson", 'w') as f:
        json.dump(to_geojson(kreis, geo), f, indent=4)


In [ ]:
import json
from data_handler import get_geojson

GEOJSON_DATA = get_geojson(1, 402)

with open("../data/kreise.geojson", 'w', encoding='utf-8') as file:
    json.dump(GEOJSON_DATA, file, ensure_ascii=False, indent=4)


<div class="image-container">
  <img src="stuttgart_map.jpeg" alt="Map of Flensburg" class="center-image" height="300">
  <img src="frankfurt_map.jpeg" alt="Map of Hamburg" class="center-image" height="300">
</div>

<style>
.image-container {
  text-align: center;
}

/* or alternatively using flexbox */
.image-container {
  display: flex;
  justify-content: center;
}
</style>

## Predicting regional supply

- The ladestation-api [](https://github.com/bundesAPI/ladestationen-api) provided by the german gouvernment provides at the moment 46,196 stations which is not the full sample but will be enough to provide major insight
- differences become clear when we look at two cities which are in the same regional context but show completly different patterns
- while in Stuttgart aproximatly 750 charging stations  available for there 600k inhabitants Frankfurt have only 222 for 750k citizen

In [ ]:
<div>
    <link rel="stylesheet" href="https://unpkg.com/leaflet@1.7.1/dist/leaflet.css" />
    <div id="mapid" style="width: 800px; height: 600px;"></div>

    <script src="https://unpkg.com/leaflet@1.7.1/dist/leaflet.js"></script>
    <script>
        // Initialize Leaflet map
        var mymap = L.map('mapid').setView([48.7758, 9.1829], 11);

        // Add a tile layer to the map (OpenStreetMap)
        L.tileLayer('https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', {
            attribution: '© OpenStreetMap contributors'
        }).addTo(mymap);

        // Load GeoJSON file and add it to the map
        fetch('stuttgart.geojson')
            .then(response => response.json())
            .then(data => {
                L.geoJSON(data).addTo(mymap);
            })
            .catch(error => {
                console.log('Error loading GeoJSON:', error);
            });
    </script>
</div>

In [ ]:
poetry run jupyter nbconvert --to=markdown src/app/app.ipynb